In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_book=pd.read_csv(r"Books.csv")
df_user=pd.read_csv(r"Users.csv")
df_rating=pd.read_csv(r"Ratings.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_13464\1590218290.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_book=pd.read_csv(r"Books.csv")


In [7]:
print(df_book.shape)
print(df_user.shape)
print(df_rating.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [19]:
df_book.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [20]:
df_user.isna().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [21]:
df_rating.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [26]:
df_user.Age.describe()

count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: Age, dtype: float64

In [34]:
x=df_user.Age.isnull()
x[x].index

Index([     0,      2,      4,      6,      7,      8,     11,     13,     14,
           15,
       ...
       278840, 278841, 278844, 278846, 278847, 278849, 278853, 278855, 278856,
       278857],
      dtype='int64', length=110762)

#### creating extra columns in df_user

In [77]:
df_user['country']=df_user.Location.apply(lambda x: x.split(",")[-1].strip().capitalize())
df_user['city']=df_user.Location.apply(lambda x: x.split(",")[0].strip().capitalize())


In [98]:
df_user.drop('Location',axis=1, inplace=True)

In [99]:
df_user

,User-ID,Age,country,city
0,1,NaN,Usa,Nyc
1,2,18.0,Usa,Stockton
2,3,NaN,Russia,Moscow
3,4,17.0,Portugal,Porto
4,5,NaN,United kingdom,Farnborough
...,...,...,...,...
278853,278854,NaN,Usa,Portland
278854,278855,50.0,United kingdom,Tacoma
278855,278856,NaN,Canada,Brampton
278856,278857,NaN,Usa,Knoxville


### imputing value of Age in df_user

In [113]:
# from sklearn.impute import KNNImputer ##### computationally heavy not recommended
# knnim=KNNImputer(n_neighbors=3,weights='uniform')##### computationally heavy not recommended
# knnim.fit_transform(df_user1)##### computationally heavy not recommended
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [150]:
lb={}
labe=LabelEncoder()
for i in df_user.select_dtypes(include=['object']):
    df_user[i+str('_label')]=labe.fit_transform(df_user[i])
    lb[i]=labe.classes_

In [151]:
df_user1=df_user[['Age','city_label','country_label']]

In [152]:
x_train=df_user1[~(pd.isnull(df_user1['Age']))][['country_label','city_label']]
y_train=df_user1[~(pd.isnull(df_user1['Age']))]['Age']
x_test=df_user1[(pd.isnull(df_user1['Age']))][['country_label','city_label']]

In [153]:
rfr=RandomForestRegressor(n_estimators=50,random_state=10,n_jobs=-1)

In [154]:
rfr.fit(x_train,y_train)

RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=10)

In [155]:
x_predict=rfr.predict(x_test)

In [156]:
df_user1.loc[pd.isnull(df_user1['Age']),'Age']=x_predict

In [161]:
df_user1['Age1']=df_user1['Age'].apply(lambda x: np.round(x,0))

In [175]:
df_user=pd.concat([df_user.drop(['country_label','city_label'], axis=1),pd.DataFrame(df_user1['Age1'])],axis=1).drop(['Age'],axis=1).rename(columns={'Age1':'Age'})

In [176]:
df_user

,User-ID,country,city,Age
0,1,Usa,Nyc,32.0
1,2,Usa,Stockton,18.0
2,3,Russia,Moscow,26.0
3,4,Portugal,Porto,17.0
4,5,United kingdom,Farnborough,32.0
...,...,...,...,...
278853,278854,Usa,Portland,38.0
278854,278855,United kingdom,Tacoma,50.0
278855,278856,Canada,Brampton,33.0
278856,278857,Usa,Knoxville,34.0


### creating a main merged dataset

In [211]:
df_user_rating=df_user.merge(df_rating, on='User-ID')

In [212]:
df_books_user_rating=df_book.merge(df_user_rating, on='ISBN')

In [213]:
df_books_user_rating[df_books_user_rating.duplicated()==True]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,country,city,Age,Book-Rating


In [214]:
df_books_user_rating.head(5) ### my main datset

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,country,city,Age,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,Usa,Stockton,18.0,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,Canada,Timmins,30.0,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,Canada,Ottawa,49.0,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,N/a,N/a,36.0,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,Canada,Sudbury,37.0,0


In [215]:
df_books_user_rating['Year-Of-Publication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

In [216]:
df_books_user_rating=df_books_user_rating[~(df_books_user_rating['Year-Of-Publication'].isin(['Gallimard','DK Publishing Inc']))]

In [217]:
df_books_user_rating['Year-Of-Publication']=df_books_user_rating['Year-Of-Publication'].astype('int')

In [218]:
df_books_user_rating.loc[(df_books_user_rating['Year-Of-Publication']==0), 'Year-Of-Publication']=2002

In [229]:
df_books_user_rating.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,country,city,Age,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,Usa,Stockton,18.0,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,Canada,Timmins,30.0,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,Canada,Ottawa,49.0,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,N/a,N/a,36.0,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,Canada,Sudbury,37.0,0


#### Popularity based recommendation engine
1. This engine is based on the popularity i.e., how popular some book , it can be based on the most read book or most rated book, etc.
2. Lets define popularity:
3. Rank(normalized(count(number of ratings))*normalized(avg_rating))
4. normlaizing formulae =x-x(min)/x(max)-x(min)

### for overall trending

In [258]:
count_of_rating_per_book=df_books_user_rating.groupby(['Book-Title'])['Book-Rating'].agg({'count'}).reset_index().rename(columns={'count':'num_of_user_rated'}).sort_values(by='num_of_user_rated', ascending=False)
count_of_rating_per_book

,Book-Title,num_of_user_rated
234948,Wild Animus,2502
196323,The Lovely Bones: A Novel,1295
183570,The Da Vinci Code,898
5303,A Painted House,838
199234,The Nanny Diaries: A Novel,828
...,...,...
147558,"Real Marriage Material (Silhouette Romance, No...",1
63663,Fabulous Fragrances II : A Guide to Prestige P...,1
147556,Real Love: The Truth About Finding Uncondition...,1
147555,Real Love: The Drawings for Sean,1


In [259]:
avg_rating=df_books_user_rating.groupby(['Book-Title'])['Book-Rating'].agg({'mean'}).reset_index().rename(columns={'mean':'avg_user_rating'}).sort_values(by='avg_user_rating', ascending=False)
avg_rating

,Book-Title,avg_user_rating
145497,Queen Victoria,10.0
19411,Bad Blood Tuskegee Syphilis Experiment,10.0
59486,Eli's Wings,10.0
59496,Elinor Fettiplace's Receipt Book: Elizabethan ...,10.0
220021,To the Elephant Graveyard,10.0
...,...,...
160438,Silent Witness (The Landry Brothers) (Intrigue...,0.0
160437,Silent Witness (Random House Large Print),0.0
74628,Ginger: My Story,0.0
74632,Gingerbread: 24 Inspirational Houses and Decor...,0.0


In [260]:
popularity_df=count_of_rating_per_book.merge(avg_rating, on='Book-Title')
popularity_df

,Book-Title,num_of_user_rated,avg_user_rating
0,Wild Animus,2502,1.019584
1,The Lovely Bones: A Novel,1295,4.468726
2,The Da Vinci Code,898,4.642539
3,A Painted House,838,3.231504
4,The Nanny Diaries: A Novel,828,3.530193
...,...,...,...
241063,"Real Marriage Material (Silhouette Romance, No...",1,0.000000
241064,Fabulous Fragrances II : A Guide to Prestige P...,1,10.000000
241065,Real Love: The Truth About Finding Uncondition...,1,0.000000
241066,Real Love: The Drawings for Sean,1,10.000000


In [261]:
popularity_df=popularity_df[popularity_df['num_of_user_rated']>=250]

In [262]:
popularity_df['num_of_user_rated_normal']=(popularity_df['num_of_user_rated']-popularity_df['num_of_user_rated'].min())/(popularity_df['num_of_user_rated'].max()-popularity_df['num_of_user_rated'].min())
popularity_df['avg_user_rating_normal']=(popularity_df['avg_user_rating']-popularity_df['avg_user_rating'].min())/(popularity_df['avg_user_rating'].max()-popularity_df['avg_user_rating'].min())

In [263]:
popularity_df['composite_score']=popularity_df['num_of_user_rated_normal']*popularity_df['avg_user_rating_normal']

In [265]:
top_50_trending=popularity_df.sort_values(by='composite_score',ascending=False).head(50)

In [269]:
top_50_trending

,Book-Title,num_of_user_rated,avg_user_rating,num_of_user_rated_normal,avg_user_rating_normal,composite_score
1,The Lovely Bones: A Novel,1295,4.468726,0.463556,0.713632,0.330808
2,The Da Vinci Code,898,4.642539,0.287111,0.749594,0.215217
6,The Secret Life of Bees,774,4.447028,0.232000,0.709143,0.164521
8,The Red Tent (Bestselling Backlist),723,4.334716,0.209333,0.685906,0.143583
4,The Nanny Diaries: A Novel,828,3.530193,0.256000,0.519449,0.132979
5,Bridget Jones's Diary,815,3.527607,0.250222,0.518914,0.129844
3,A Painted House,838,3.231504,0.260444,0.457649,0.119192
23,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453,0.135111,0.861510,0.116400
10,Life of Pi,664,4.088855,0.183111,0.635037,0.116282
18,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652,0.143556,0.801964,0.115126


In [272]:
top_50_trending=top_50_trending.merge(df_book,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_of_user_rated','avg_user_rating']]

#### collaborative filtering based
1. This is basically a distance based approach in which each product (in our case book) needs to be consider as a point in a plane with columns as all respective user rating.
2. Then for each book distance is calculated between the vectors.

In [288]:
temp=df_books_user_rating.groupby(['User-ID'])['Book-Rating'].agg({'count'})['count']>200
temp[temp].index
users_given_morethan200reviews=temp[temp].index

In [291]:
df_books_user_rating_morethan200_rating=df_books_user_rating[df_books_user_rating['User-ID'].isin(users_given_morethan200reviews)]

In [294]:
df_books_user_rating_morethan200_rating.drophead()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,country,city,Age,Book-Rating
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,N/a,N/a,36.0,8
6,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,85526,Canada,Victoria,36.0,0
7,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,96054,Canada,Ottawa,29.0,0
10,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,177458,Canada,Ottawa,29.0,0
21,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,110912,Usa,Milpitas,36.0,10


In [299]:
## now looking from books side looking for books having more than 50 rating
books_for_morethan50=df_books_user_rating_morethan200_rating.groupby(['Book-Title'])['Book-Rating'].agg({'count'})['count']>50
books_for_morethan50_reviews=books_for_morethan50[books_for_morethan50].index

In [301]:
df_books_user_rating_morethan200_rating_50_reviews=df_books_user_rating_morethan200_rating[df_books_user_rating_morethan200_rating['Book-Title'].isin(books_for_morethan50_reviews)]

In [306]:
pivot_df_books_user_rating_morethan200_rating_50_reviews=df_books_user_rating_morethan200_rating_50_reviews.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating').fillna(0)

In [309]:
from sklearn.metrics.pairwise import cosine_similarity

In [315]:
similarity_among_book_distance=cosine_similarity(pivot_df_books_user_rating_morethan200_rating_50_reviews)

In [382]:
def recommend(book_name):
    index=np.where(pivot_df_books_user_rating_morethan200_rating_50_reviews.index==book_name)[0][0]
    similarity_score=sorted(list(enumerate(similarity_among_book_distance[index])),key=lambda x:x[1],reverse=True)[1:5]
    data=[]
    for i in similarity_score:
        item=[]
        temp_df=df_books_user_rating[df_books_user_rating['Book-Title']==pivot_df_books_user_rating_morethan200_rating_50_reviews.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        data.append(item)
    return data
        
        

In [383]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg']]